In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mnist_example import MNISTModel, MNISTDataModule

datamodule = MNISTDataModule(batch_size=10)
datamodule.setup()

len(datamodule.test_dataloader())

10

In [3]:
from energizer.data.datamodule import ActiveDataModule

dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

dm.is_on_pool = False
print(len(dm.test_dataloader()))
dm.is_on_pool = True
print(len(dm.test_dataloader()))

10
10


In [ ]:
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )
# dm.label(list(range(100)))
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )

---

In [11]:
from energizer.learners.base import Learner
from energizer.trainer import Trainer
import torch
from torch.utils.data import DataLoader
from energizer.learners.base import Deterministic
from energizer.learners.acquisition_functions import entropy
from pytorch_lightning import LightningModule, Trainer as pl_Trainer

In [12]:
class ActiveLearner(Deterministic):

    def pool_step(self, batch, *args, **kwargs) -> torch.Tensor:
        x, _ = batch
        logits = self(x)
        scores = entropy(logits)
        return scores

In [33]:
model = MNISTModel()
active_learner = ActiveLearner(model)

In [34]:
pl_trainer = pl_Trainer(max_epochs=1)
pl_trainer.fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 20/20 [00:00<00:00, 68.94it/s, loss=2.33, v_num=323]


In [36]:
pl_trainer.test(active_learner, datamodule)

Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 219.49it/s]


[{}]

In [32]:
trainer = Trainer(query_size=2, max_epochs=1, max_labelling_iters=2)
trainer.active_fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:193

current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2934, 2.2930, 2.2889, 2.2925, 2.2914, 2.2890, 2.2933,
        2.2952, 2.2921, 2.2934])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2952, 2.2934])
top_indices: tensor([7, 0]) 

current batch_size: 10
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
size: tensor(20)
all_scores: tensor([2.2952, 2.2934, 2.2928, 2.2943, 2.2911, 2.2969, 2.2972, 2.2919, 2.2958,
        2.2960, 2.2938, 2.2960])
all_indices: tensor([ 7,  0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
top_scores: tensor([2.2972, 2.2969])
top_indices: tensor([14, 13]) 

current batch_size: 10
current_indices: tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
size: tensor(30)
all_scores: tensor([2.2972, 2.2969, 2.2882, 2.2910, 2.2942, 2.2876, 2.2946, 2.2809, 2.2925,
        2.2930, 2.2957, 2.2938])
all_indices: tensor([14, 13, 20, 21, 22, 

...

Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 224.09it/s]
current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2889, 2.2890, 2.2882, 2.2910, 2.2876, 2.2809, 2.2902,
        2.2888, 2.2840, 2.2898])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2910, 2.2902])
top_indices: tensor([3, 6]) 

current batch_size: 10
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
size: tensor(20)
all_scores: tensor([2.2910, 2.2902, 2.2872, 2.2887, 2.2910, 2.2863, 2.2848, 2.2878, 2.2900,
        2.2855, 2.2897, 2.2858])
all_indices: tensor([ 3,  6, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
top_scores: tensor([2.2910, 2.2910])
top_indices: tensor([12,  3]) 

current batch_size: 8
current_indices: tensor([20, 21, 22, 23, 24, 25, 26, 27])
size: tensor(28)
all_scores: tensor([2.2910, 2.2910, 2.2865, 2.2872, 2.2864, 2.2902, 2.2885, 2.2880, 2.2903,
        2.28

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2889, 2.2890, 2.2882, 2.2876, 2.2809, 2.2888, 2.2840,
        2.2872, 2.2887, 2.2863])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2890, 2.2889])
top_indices: tensor([1, 0]) 

current batch_size: 10
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
size: tensor(20)
all_scores: tensor([2.2890, 2.2889, 2.2848, 2.2878, 2.2855, 2.2858, 2.2865, 2.2872, 2.2864,
        2.2885, 2.2880, 2.2889])
all_indices: tensor([ 1,  0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
top_scores: tensor([2.2890, 2.2889])
top_indices: tensor([ 1, 19]) 

compute indices: tensor([ 1, 19])
Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 217.42it/s]
current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2889, 2.2882, 2.2876, 2.2809, 2.2

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 200.15it/s]
current batch_size: 8
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7])
size: tensor(8)
all_scores: tensor([  -inf,   -inf, 2.2809, 2.2840, 2.2863, 2.2848, 2.2855, 2.2858, 2.2865,
        2.2864])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7])
top_scores: tensor([2.2865, 2.2864])
top_indices: tensor([6, 7]) 

compute indices: tensor([6, 7])
Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 173.73it/s]
current batch_size: 6
current_indices: tensor([0, 1, 2, 3, 4, 5])
size: tensor(6)
all_scores: tensor([  -inf,   -inf, 2.2809, 2.2840, 2.2863, 2.2848, 2.2855, 2.2858])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5])
top_scores: tensor([2.2863, 2.2858])
top_indices: tensor([2, 5]) 

compute indices: tensor([2, 5])
Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 224.51it/s]
current batch_size: 4
current_indices: tensor([0, 1, 2, 3])
size: tensor(4)
all_scores: tensor([  -inf,   -in

In [ ]:
is_overridden("pool_epoch_end", active_learner)

In [ ]:
trainer.test(model, datamodule)

In [28]:
test_dl = DataLoader(
    [datamodule.mnist_test[i] for i in range(10)], 
    shuffle=False, 
    batch_size=2,
)
x, y = next(iter(test_dl))

In [30]:
active_learner(x).argmax(-1)

tensor([8, 6])

In [31]:
y

tensor([7, 2])